### 文本关键词提取模型

- 数据清洗：特殊字符过滤，错误数据更正
- 关键词特征提取：文本基础特征，关键词间交互特征，统计特征等

In [1]:
import os
print (os.path.abspath('.'))

E:\PythonNotebook\Top方案\文本关键词抽取


In [1]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from sklearn.preprocessing import MinMaxScaler
import re
import pickle
from operator import itemgetter
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import gc
import math
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, f1_score
from scipy.stats import skew, kurtosis
from collections import Counter
from tqdm import tqdm

%matplotlib inline

pd.options.display.max_rows = 700

### 添加自定义词典

In [2]:
## 搜狗+百度词典 深蓝词典转换
jieba.load_userdict('./字典/明星.txt')
jieba.load_userdict('./字典/实体名词.txt')
jieba.load_userdict('./字典/歌手.txt')
jieba.load_userdict('./字典/动漫.txt')
jieba.load_userdict('./字典/电影.txt')
jieba.load_userdict('./字典/电视剧.txt')
jieba.load_userdict('./字典/流行歌.txt')
jieba.load_userdict('./字典/创造101.txt')
jieba.load_userdict('./字典/百度明星.txt')
jieba.load_userdict('./字典/美食.txt')
jieba.load_userdict('./字典/FIFA.txt')
jieba.load_userdict('./字典/NBA.txt')
jieba.load_userdict('./字典/网络流行新词.txt')
jieba.load_userdict('./字典/显卡.txt')

## 爬取漫漫看网站和百度热点上面的词条
jieba.load_userdict('./字典/漫漫看_明星.txt')
jieba.load_userdict('./字典/百度热点人物+手机+软件.txt')
jieba.load_userdict('./字典/自定义词典.txt')

## 实体名词抽取之后的结果 有一定的人工过滤 
## origin_zimu 这个只是把英文的组织名过滤出来
jieba.load_userdict('./字典/person.txt')
jieba.load_userdict('./字典/origin_zimu.txt')

## 第一个是所有《》里面出现的实体名词
## 后者是本地测试集的关键词加上了 
jieba.load_userdict('./字典/出现的作品名字.txt')
jieba.load_userdict('./字典/val_keywords.txt')

## 网上随便找的停用词合集
jieba.analyse.set_stop_words('./stopword.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.507 seconds.
Prefix dict has been built succesfully.


### 读取数据

In [3]:
all_docs = pd.read_csv('all_docs.txt', sep='\001', header=None)
all_docs.columns = ['id', 'title', 'content']
all_docs.fillna('', inplace=True)

val = pd.read_csv('train_docs_keywords.txt', sep='\t', header=None)
val.columns = ['id', 'kw']
val.kw = val.kw.apply(lambda x: x.split(','))

In [4]:
all_docs = pd.read_csv('all_docs.txt', sep='\001', header=None)
all_docs.columns = ['id', 'title', 'content']
all_docs.fillna('', inplace=True)
all_docs.head()

,id,title,content
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...


文本关键词标签

In [5]:
val = pd.read_csv('train_docs_keywords.txt', sep='\t', header=None)
val.columns = ['id', 'kw']
val.kw = val.kw.apply(lambda x: x.split(','))

In [6]:
val.head()

,id,kw
0,D012650,"[三生三世, 凤九, 东华]"
1,D047118,"[南宁, 美食]"
2,D098970,"[凉菜, 北京人]"
3,D092010,"[华为, P30pro]"
4,D103408,"[酒吧, 世界杯]"


### 数据清洗

In [7]:
all_docs['title_cut'] = all_docs['title'].apply(lambda x:''.join(filter(lambda ch: ch not in ' \t◆#%', x)))
all_docs['content_cut'] = all_docs['content'].apply(lambda x:''.join(filter(lambda ch: ch not in ' \t◆#%', x)))

all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&amp;', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&quot;', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&#34;', ' ', x))

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&amp;', ' ', x))
all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&quot;', ' ', x))
all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&#34;', ' ', x))

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&nbsp;', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&nbsp;', ' ', x))

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&gt;', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&gt;', ' ', x))

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('&lt;', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('&lt;', ' ', x))

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub('hr/', ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub('hr/', ' ', x))

strinfo = re.compile('······')

all_docs['content_cut'] = all_docs['content_cut'].apply(lambda x: re.sub(strinfo, ' ', x))
all_docs['title_cut'] = all_docs['title_cut'].apply(lambda x: re.sub(strinfo, ' ', x))

In [8]:
all_docs.head()

,id,title,content,title_cut,content_cut
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...


### 句子拆分
- 目的是考虑词的位置，加大关键词在标题/首局/末句的词频（权重）

In [9]:
def split_sentences(x):

    ret = pd.Series()

    ret['id'] = x['id']
    if x['content_cut'] == '' or len(x['content_cut']) < 2:
        ret['first_sentence'] = ''
        ret['other_sentence'] = ''
        ret['last_sentence'] = ''
        return ret

    sentence_delimiters = re.compile(u'[。？！；!?]')
    sentences =[i for i in sentence_delimiters.split(x['content_cut']) if i != '']
    num_sen = len(sentences)

    if num_sen == 1:
        ret['first_sentence'] = sentences[0]
        ret['other_sentence'] = ''
        ret['last_sentence'] = sentences[0]
    elif num_sen == 2:
        ret['first_sentence'] = sentences[0]
        ret['other_sentence'] = ''
        ret['last_sentence'] = sentences[-1]
    else:
        ret['first_sentence'] = sentences[0]
        ret['other_sentence'] = ''.join(sentences[1:-1])
        ret['last_sentence'] = sentences[-1]
    return ret

In [10]:
tmp = all_docs.apply(split_sentences, axis=1)

all_docs = pd.merge(all_docs, tmp, on='id', how='left')

In [11]:
all_docs.head()

,id,title,content,title_cut,content_cut,first_sentence,other_sentence,last_sentence
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,可能这么些年过去，心态不断调整过了，至少在微博上，陈若仪越来越放得开，晒自己带娃照顾双子星的...,毕竟像她另一个爱用的达尔肤面膜，效果好是好，价格据说比sk2都还要贵，不是大多数人日常能够消...
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,翩若惊鸿，婉若游龙,曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象直到在《大军师司马懿》里看到张钧甯...,瓌姿艳逸，仪静体闲
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的蓝洁瑛算得上一代佳丽，一张妩媚动人的脸蛋美得让人着迷，早在1984年她因出演电视剧《...,无论蓝洁瑛选择了怎样的生活方式，都希望她能够过得幸福美满，最后祝她生日快乐
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,"""林心如还在活动中分享了不少女儿之间的趣事随后，【霍建华会向女儿索吻】登上热搜榜却使霍建华的...",但霍建华婚后这副有女万事足的样子，还是让人深信他结婚是幸福和开心的
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的,可以说对于那个时期的TVB演员相当熟悉，那又有多少男观众曾经将女艺人视为女神呢今天要说的这位...,发出来更多的是感概下当时有看她年轻模样的观众，其实我们也和她一起变老了


In [12]:
# 取出第一句的作品名字 《。。。》
# 假如标题中存在《。。。》，那么里面的内容直接就是关键词

all_docs['first_sentence_reg'] = all_docs['first_sentence'].apply(lambda x:re.findall(r"《(.+?)》",x))
all_docs.head()

,id,title,content,title_cut,content_cut,first_sentence,other_sentence,last_sentence,first_sentence_reg
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,可能这么些年过去，心态不断调整过了，至少在微博上，陈若仪越来越放得开，晒自己带娃照顾双子星的...,毕竟像她另一个爱用的达尔肤面膜，效果好是好，价格据说比sk2都还要贵，不是大多数人日常能够消...,[爸爸去哪儿]
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,翩若惊鸿，婉若游龙,曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象直到在《大军师司马懿》里看到张钧甯...,瓌姿艳逸，仪静体闲,[]
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的蓝洁瑛算得上一代佳丽，一张妩媚动人的脸蛋美得让人着迷，早在1984年她因出演电视剧《...,无论蓝洁瑛选择了怎样的生活方式，都希望她能够过得幸福美满，最后祝她生日快乐,[大话西游之月光宝盒]
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,"""林心如还在活动中分享了不少女儿之间的趣事随后，【霍建华会向女儿索吻】登上热搜榜却使霍建华的...",但霍建华婚后这副有女万事足的样子，还是让人深信他结婚是幸福和开心的,[]
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的,可以说对于那个时期的TVB演员相当熟悉，那又有多少男观众曾经将女艺人视为女神呢今天要说的这位...,发出来更多的是感概下当时有看她年轻模样的观众，其实我们也和她一起变老了,[]


In [13]:
all_docs.to_csv('prepross_all_docs.csv')

In [14]:
all_docs = pd.read_csv('prepross_all_docs.csv')

In [16]:
all_docs = all_docs[['id','title','content','title_cut','content_cut','first_sentence','other_sentence','last_sentence','first_sentence_reg']]

In [31]:
## 历史作品名字 作为下面的一个特征
TV = []
with open('./字典/出现的作品名字.txt', 'r', encoding='utf-8') as f:
    for word in f.readlines():
        TV.append(word.strip())

In [17]:
## 这是根据本数据集算的idf文件
idf = {}
with open('my_idf.txt', 'r', encoding='utf-8') as f:
    for i in f.readlines():
        if len(i.strip().split()) == 2:
            v = i.strip().split()
            idf[v[0]] = float(v[1])

In [18]:
idf

{'公众': 3.4555108742322953,
 '喷': 4.952738430045061,
 '。': -0.0,
 '面前': 3.141560874959904,
 '按摩': 4.6938997295416085,
 '几个': 2.4167618223540868,
 '越来越': 2.2850564411800547,
 '怎么弄': 7.981696351227372,
 '这么些': 7.515076819965877,
 '防晒': 5.304755703709812,
 '敷起': 10.494001975203487,
 '到': 0.6095783228895851,
 '自己': 0.742985957492428,
 '林志颖': 6.622800964295596,
 '格外': 4.079996719197085,
 '半': 3.942921640160082,
 '还是': 0.9391031011338262,
 '可能': 1.3526544747142555,
 '双子星': 7.90373480975766,
 '隔离': 5.549981430189215,
 '遗传': 4.642757808870824,
 '平滑': 6.259895470606227,
 '地说': 4.342688727153608,
 '至少': 2.895602645879523,
 '后天': 5.412597610219024,
 '经过': 2.3987081983588383,
 '当时': 2.193721785350823,
 '放得开': 7.515076819965877,
 '吸引住': 6.694774463920685,
 '呵护': 4.99956972972916,
 '店': 3.9096710939933046,
 '钱': 3.0443100774303065,
 '老婆': 3.9709292651469856,
 '七八次': 8.596881990317605,
 '这个': 0.9028763300133684,
 '一天': 2.9839363823331806,
 '之类': 3.903243156341906,
 '膝盖': 5.080868934226869,
 '又': 0.907

### Doc2Vec 聚类

In [20]:
from gensim.models import Doc2Vec
from gensim.models import Word2Vec

In [21]:
## classes_doc2vec.npy 文件是先算DOC2VEC向量 然后用Kmeans简单聚成10类
classes = np.load('classes_doc2vec.npy')

all_docs['classes'] = classes

In [22]:
all_docs.head()

,id,title,content,title_cut,content_cut,first_sentence,other_sentence,last_sentence,first_sentence_reg,classes
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,可能这么些年过去，心态不断调整过了，至少在微博上，陈若仪越来越放得开，晒自己带娃照顾双子星的...,毕竟像她另一个爱用的达尔肤面膜，效果好是好，价格据说比sk2都还要贵，不是大多数人日常能够消...,['爸爸去哪儿'],2
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,翩若惊鸿，婉若游龙,曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象直到在《大军师司马懿》里看到张钧甯...,瓌姿艳逸，仪静体闲,[],1
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,年轻时的蓝洁瑛算得上一代佳丽，一张妩媚动人的脸蛋美得让人着迷，早在1984年她因出演电视剧《...,无论蓝洁瑛选择了怎样的生活方式，都希望她能够过得幸福美满，最后祝她生日快乐,['大话西游之月光宝盒'],1
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,"""林心如还在活动中分享了不少女儿之间的趣事随后，【霍建华会向女儿索吻】登上热搜榜却使霍建华的...",但霍建华婚后这副有女万事足的样子，还是让人深信他结婚是幸福和开心的,[],2
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的,可以说对于那个时期的TVB演员相当熟悉，那又有多少男观众曾经将女艺人视为女神呢今天要说的这位...,发出来更多的是感概下当时有看她年轻模样的观众，其实我们也和她一起变老了,[],1


## Doc2vec word2vec 计算主题相似性
将Doc2vec word2vec两个模型的向量长度定为一样长就可以直接计算余弦相似度、欧式距离等

In [23]:
doc2vec_model = Doc2Vec.load('doc2vec.model')

word2vec_model = Word2Vec.load('word2vec.model')

In [24]:
wv= word2vec_model.wv

In [25]:
def Cosine(vec1, vec2):
    npvec1, npvec2 = np.array(vec1), np.array(vec2)
    return npvec1.dot(npvec2)/(math.sqrt((npvec1**2).sum()) * math.sqrt((npvec2**2).sum()))


def Euclidean(vec1, vec2):
    npvec1, npvec2 = np.array(vec1), np.array(vec2)
    return math.sqrt(((npvec1-npvec2)**2).sum())

In [26]:
## 后面加大窗口和迭代又算了一次word2vec 模型 主要是用来算候选关键词之间的相似度

word2vec_model_256 = Word2Vec.load('word2vec_iter10_sh_1_hs_1_win_10.model')

In [27]:
all_docs['idx'] = all_docs.index.values

### 关键词特征提取

In [2]:
def get_train_df(df, train=True):
    res = []
    for index in tqdm(df.index):
        #遍历获取每一条数据
        x = df.loc[index]
        # TF-IDF候选关键词结果
        first_sentence_reg = ' '.join(x['first_sentence_reg'])
        
        # 对文章进行手动加权
        text = 19*(str(x['title_cut'])+'。')+ 3*(str(x['first_sentence'])+'。') + 1*(str(x['other_sentence'])+'。')+\
                3*(str(x['last_sentence'])+ '。') + 7*str(first_sentence_reg)
        # TF-IDF候选关键词结果
        # 注：原作者更改了结巴分词中的代码，参数allowpPOS实际是不允许出现的词性 即allowPOS = NotAllowPOS
        jieba_tags = jieba.analyse.extract_tags(sentence=text, topK=20, allowPOS=('r','m','d', 'p', 'q', 'ad', 'u', 'f'), withWeight=True,\
                                          withFlag=True)
        #获得extract_tags中所有结果
        tags = []
        cixing = []
        weight = []
        len_tags = []
        for tag in jieba_tags:
            tags.append(tag[0].word)
            cixing.append(tag[0].flag)
            weight.append(tag[1])
        
        # 分隔符：用来切分句子用
        sentence_delimiters = re.compile(u'[。？！；!?]')
        # 切分句子
        sentences =[i for i in sentence_delimiters.split(text) if i != '']
        # 包含句子数量
        num_sen = len(sentences)
        # 获得当前文本所有词语
        words = []
        num_words = 0
        for sen in sentences:
            cut = jieba.lcut(sen)
            words.append(cut)
            num_words += len(cut)
        for i in range(len(tags)):
            len_tags.append(len(tags[i]))
            
        new_tags = tags
        new_weight = cixing
        new_cixing = weight
        
        
            
        ## 位置特征： 1. 是否出现在标题 2.是否出现在第一句 3.是否出现在最后一句 4.出现在正文中间部分
        occur_in_title = np.zeros(len(new_tags))
        occur_in_first_sentence = np.zeros(len(new_tags))
        occur_in_last_sentence = np.zeros(len(new_tags))
        occur_in_other_sentence = np.zeros(len(new_tags))
        for i in range(len(new_tags)):
            if new_tags[i] in x['title_cut']:
                occur_in_title[i] = 1
            if new_tags[i] in x['first_sentence']:
                occur_in_first_sentence[i] = 1
            if new_tags[i] in x['last_sentence']:
                occur_in_last_sentence[i] = 1
            try:
                if new_tags[i] in x['other_sentence']:
                    occur_in_other_sentence[i] = 1
            except:
                occur_in_other_sentence[i] = 1
                print (new_tags[i])

                    
        
        
        ## 共现矩阵及相关统计特征,例如均值、方差、偏度等 
        num_tags = len(new_tags)
        #初始化共现矩阵
        arr = np.zeros((num_tags, num_tags))
        # 计算出现个数
        for i in range(num_tags):
            for j in range(i+1, num_tags):
                count = 0
                for word in words:
                    if new_tags[i] in word and new_tags[j] in word:
                        count += 1
                arr[i, j] = count
                arr[j, i] = count
        # 计算统计特征
        ske = stats.skew(arr)
        var_gongxian = np.zeros(len(new_tags))
        kurt_gongxian = np.zeros(len(new_tags))
        diff_min_gongxian = np.zeros(len(new_tags))
        for i in range(len(new_tags)):
            var_gongxian[i] = np.var(arr[i])
            kurt_gongxian[i] = stats.kurtosis(arr[i])
            diff_sim = np.diff(arr[i])
            if len(diff_sim) > 0:
                diff_min_gongxian[i] = np.min(diff_sim)

                
        ## textrank特征，跟pagerank原理类似，如果一个单词出现在很多单词后面的话，那么说明这个单词比较重要，
        ## 一个TextRank值很高的单词后面跟着的一个单词，那么这个单词的TextRank值会相应地因此而提高
        textrank_tags = dict(jieba.analyse.textrank(sentence=text, allowPOS=('r','m','d', 'p', 'q', 'ad', 'u', 'f'), withWeight=True))
        
        textrank = []
        for tag in new_tags:
            if tag in textrank_tags:
                textrank.append(textrank_tags[tag])
            else:
                textrank.append(0)
        # 得到所有词，list格式
        all_words = np.concatenate(words).tolist()
        
        ## 词频
        tf = []
        for tag in new_tags:
            tf.append(all_words.count(tag))
        tf = np.array(tf)
        
        ## hf: 头词频，文本内容前1/4候选词词频
        hf = []
        head = len(words) // 4 + 1
        head_words = np.concatenate(words[:head]).tolist()
        for tag in new_tags:
            hf.append(head_words.count(tag))
        
        ## has_num：是否包含数字
        ## has_eng: 是否包含字母
        def hasNumbers(inputString):
            return bool(re.search(r'\d', inputString))
        def hasEnglish(inputString):
            return bool(re.search(r'[a-zA-Z]', inputString))
        has_num = []
        has_eng = []
        for tag in new_tags:
            if hasNumbers(tag):
                has_num.append(1)
            else:
                has_num.append(0)
            if hasEnglish(tag):
                has_eng.append(1)
            else:
                has_eng.append(0)
                
        ## is_TV:是否为作品名称
        is_TV = []
        for tag in new_tags:
            if tag in TV:
                is_TV.append(1)
            else:
                is_TV.append(0)
                
        ## idf: 用训练集跑出的逆词频
        v_idf = []
        for tag in new_tags:
            v_idf.append(idf.get(tag, 0)) 
        
        ## 计算文本相似度，这里直接用doc2vec跟每个单词的word2vec做比较
        ## sim: 余弦相似度
        ## sim_euc：欧氏距离
        default = np.zeros(100)
        doc_vec = doc2vec_model.docvecs.vectors_docs[x['idx']]
        sim = []
        sim_euc = []
        for tag in new_tags:
            if tag in wv:
                sim.append(Cosine(wv[tag], doc_vec))
                sim_euc.append(Euclidean(wv[tag], doc_vec))
            else:
                sim.append(Cosine(default, doc_vec))
                sim_euc.append(Euclidean(default, doc_vec))
                
        ## 关键词所在句子长度，记录为列表，然后算统计特征 
        mean_l2 = np.zeros(len(new_tags))
        max_l2 = np.zeros(len(new_tags))
        min_l2 = np.zeros(len(new_tags))
        for i in range(len(new_tags)):
            tmp = []
            for word in words:
                if new_tags[i] in word:
                    tmp.append(len(word))
            if len(tmp) > 0:
                mean_l2[i] = np.mean(tmp)
                max_l2[i] = np.max(tmp)
                min_l2[i] = np.min(tmp)
                
        ## 关键词所在位置，记录为列表，然后算统计特征 
        min_pos = [np.NaN for _ in range(len(new_tags))]
        diff_min_pos_bili = [np.NaN for _ in range(len(new_tags))]
        diff_kurt_pos_bili = [np.NaN for _ in range(len(new_tags))]
        
        for i in range(len(new_tags)):
            # 得到当前候选标签出现的所有位置
            pos = [a for a in range(len(all_words)) if all_words[a] == new_tags[i]]
            # 计算相对位置
            pos_bili = np.array(pos) / len(all_words)
            
            if len(pos) > 0:
                min_pos[i] = np.min(pos)
                # 差分特征
                diff_pos = np.diff(pos)
                diff_pos_bili = np.diff(pos_bili)
                if len(diff_pos) > 0:
                    diff_min_pos_bili[i] = np.min(diff_pos_bili)
                    diff_kurt_pos_bili[i] = stats.kurtosis(diff_pos_bili)   
        ## 候选关键词之间的相似度 word2vec gensim 窗口默认 迭代默认 向量长度100
        ## sim_tags_arr：相似度矩阵
        sim_tags_arr = np.zeros((len(new_tags), len(new_tags)))
        for i in range(len(new_tags)):
            for j in range(i+1, len(new_tags)):
                if new_tags[i] in wv and new_tags[j] in wv:
                    sim_tags_arr[i, j] = word2vec_model.similarity(new_tags[i], new_tags[j])
                    sim_tags_arr[j, i] = sim_tags_arr[i, j]
        # 计算当前次和其他词平均结果
        mean_sim_tags = np.zeros(len(new_tags))
        diff_mean_sim_tags = np.zeros(len(new_tags))     
        for i in range(len(new_tags)):
            mean_sim_tags[i] = np.mean(sim_tags_arr[i])
            diff_sim = np.diff(sim_tags_arr[i])
            if len(diff_sim) > 0:
                diff_mean_sim_tags[i] = np.mean(diff_sim)

        ## 候选关键词之间的相似度 word2vec gensim 窗口10 迭代10 向量长度256 
    
        sim_tags_arr_255 = np.zeros((len(new_tags), len(new_tags)))
        for i in range(len(new_tags)):
            for j in range(i+1, len(new_tags)):
                if new_tags[i] in word2vec_model_256 and new_tags[j] in word2vec_model_256:
                    sim_tags_arr_255[i, j] = word2vec_model_256.similarity(new_tags[i], new_tags[j])
                    sim_tags_arr_255[j, i] = sim_tags_arr_255[i, j]
        ## label 训练集打标签
        if train:
            label = []
            for tag in new_tags:
                if tag in x.kw:
                    label.append(1)
                else:
                    label.append(0)
                    
        ## 不同词性的比例
        cixing_counter = Counter(new_cixing)
        
        fea = pd.DataFrame()
        fea['id'] = [x['id'] for _ in range(len(new_tags))]
        fea['tags'] = new_tags
        fea['cixing'] = new_cixing


        fea['tfidf'] = new_weight
        fea['ske'] = ske
        
        fea['occur_in_title'] = occur_in_title
        fea['occur_in_first_sentence'] = occur_in_first_sentence
        fea['occur_in_last_sentence'] = occur_in_last_sentence
        fea['occur_in_other_sentence'] = occur_in_other_sentence
        fea['len_tags'] = len_tags
        fea['num_tags'] = num_tags
        fea['num_words'] = num_words
        fea['num_sen'] = num_sen
        fea['classes'] = x['classes']

        fea['len_text'] = len(x['title_cut'] + x['content_cut'])
        fea['textrank'] = textrank
        fea['word_count'] = tf
        fea['tf'] = tf / num_words
        fea['num_head_words'] = len(head_words)
        fea['head_word_count'] = hf
        fea['hf'] = np.array(hf) / len(head_words)
        fea['pr'] = tf / tf.sum()
        fea['has_num'] = has_num
        fea['has_eng'] = has_eng
        fea['is_TV'] = is_TV
        fea['idf'] = v_idf
        fea['sim'] = sim
        fea['sim_euc'] = sim_euc

        fea['mean_l2'] = mean_l2
        fea['meaxl2'] = max_l2
        fea['min_l2'] = min_l2
        
        fea['min_pos'] = min_pos
        fea['diff_min_pos_bili'] = diff_min_pos_bili
        fea['diff_kurt_pos_bili'] = diff_kurt_pos_bili
    
        #fea['diff_max_min_sen_pos'] = diff_max_min_sen_pos
        #fea['diff_var_sen_pos_bili'] = diff_var_sen_pos_bili

        fea['mean_sim_tags'] = mean_sim_tags
        fea['diff_mean_sim_tags'] = diff_mean_sim_tags

        #fea['kurt_sim_tags_256'] = kurt_sim_tags_256
        #fea['diff_max_min_sim_tags_256'] = diff_max_min_sim_tags_256
        fea['var_gongxian'] = var_gongxian
        fea['kurt_gongxian'] = kurt_gongxian
        fea['diff_min_gongxian'] = diff_min_gongxian
        
        ## 当前文本候选关键词词性比例
        for c in ['x', 'nz', 'l', 'n', 'v', 'ns', 'j', 'a', 'vn', 'nr', 'eng', 'nrt',
                  't', 'z', 'i', 'b', 'o', 'nt', 'vd', 'c', 's', 'nrfg', 'mq', 'rz',
                  'e', 'y', 'an', 'rr']:
            fea['cixing_{}_num'.format(c)] = cixing_counter[c]
            fea['cixing_{}_bili'.format(c)] = cixing_counter[c] / (len(new_cixing) + 1)

        if train:
            fea['label'] = label
        res.append(fea)
    return res

In [29]:
train_doc = pd.merge(val, all_docs, on='id', how='left')

In [ ]:
%%time
res = get_train_df(train_doc, train=True)

train_df = pd.concat(res, axis=0).reset_index(drop=True)

In [ ]:
%%time
res = get_train_df(all_docs, train=False)

test_df = pd.concat(res, axis=0).reset_index(drop=True)


  0%|          | 0/108295 [00:00<?, ?it/s]c:\anaconda3\lib\site-packages\ipykernel_launcher.py:240: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
c:\anaconda3\lib\site-packages\ipykernel_launcher.py:280: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\anaconda3\lib\site-packages\ipykernel_launcher.py:281: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).

  0%|          | 2/108295 [00:00<7:48:42,  3.85it/s] 
Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\anaconda3\lib\site-packages\tqdm\_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "c:\anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data

In [9]:
train_df = pd.to_csv('train_df_v7.csv', index=False)
test_df = pd.to_csv('train_df_v7.csv', index=False)